### <div class="subsection_title">Prequels/sequels

- [Kaggle Machine Learning & Data Science (data-prep)](https://www.kaggle.com/neomatrix369/kaggle-machine-learning-data-science-data-prep/) | [Extended Dataset](https://www.kaggle.com/neomatrix369/kaggle-machine-learning-data-science-survey-ext) | [Additional Dataset](https://www.kaggle.com/neomatrix369/world-bank-data-1960-to-2016-extended)
- **Kaggle Global Outreach (analysis)**

In [ ]:
"""
CSS/HTML settings to allow some nice things like setting styles on 
section, sub-section, sub-sub-section, and sub-sub-sub-section headers.

All of this is possible with the help of the HTML and Javascript magic from IPython.core.display
"""
from IPython.core.display import display, HTML, Javascript

html_css_settings = """
<!DOCTYPE html>
<html lang="en">
    <head>
    <style>
    .toc h2{
        color: #3f4d63;
        font-weight: 600;
        font-family: 'Times New Roman', serif;
        font-size: 28px;
        margin-bottom: 4px;
    }
    
    .toc ol li{
        list-style:none;
        line-height:normal;
        }
     
    .toc li{
        color: #235f83;
        font-weight: 600;
        font-family: 'Times New Roman', serif;
        font-size: 17px;
        margin-bottom: 2px;
    }

    .toc ol ol li{
        color: #4d4d4d;
        font-weight: 400;
        font-size: 15px;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        margin-top: 0px;
        margin-bottom: 0px;
    } 
    
    .section_title{
        background-color: #3f4d63;
        color: white;
        font-family: Helvetica;
        font-size: 25px;
        padding: 6px 12px;
        margin-bottom: 5px;
    }
    .subsection_title{
        background: #235f83;
        color: white;
        font-family: Helvetica;
        font-size: 21px;
        padding: 6px 12px;
        margin-bottom: 0px;
    }
    .subsubsection_title{
        background: #235f83;
        color: white;
        font-family: Helvetica;
        font-size: 17px;
        padding: 6px 12px;
        margin-bottom: 0px;
    }
    .subsubsubsection_title{
        background: #235f83;
        color: white;
        font-family: Helvetica;
        font-size: 15px;
        padding: 6px 12px;
        margin-bottom: 0px;
    }
    </style>
    </head>
    <body>
    </body>
</html>
"""

HTML(html_css_settings)

In [ ]:
!pip install -U ipywidgets
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numexpr
import math

import warnings
warnings.filterwarnings('ignore')

import missingno as msno

import plotly.express as px
import plotly.graph_objects as go

from plotly.offline import init_notebook_mode, iplot
from plotly import subplots
import plotly.figure_factory as ff
import plotly as py
import plotly.graph_objects as go
init_notebook_mode(connected=True)

import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML

import gc
import random

# Draw maps using folium (https://python-visualization.github.io/folium/)
import folium
from folium.features import DivIcon
from branca.colormap import LinearColormap

# Make interactive widgets on the notebook to interact with components
# like maps and other displayable items
import ipywidgets as widgets
from ipywidgets import Layout, IntSlider, jslink, HBox, interactive_output
from folium.plugins import FastMarkerCluster

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# Colab sets some Seaborn styles by default; let's revert to the default
# Matplotlib styles and plot again.
plt.rcdefaults()

sns.set(style="whitegrid", font_scale=1.75)

# prettify plots
plt.rcParams['figure.figsize'] = [20.0, 5.0]
plt.rcParams['figure.dpi'] = 200
sns.set_palette(sns.color_palette("muted"))

%matplotlib inline

# Increase the quality and resolution of our charts so we can copy/paste or just
# directly save from here.
# See:
# https://ipython.org/ipython-doc/3/api/generated/IPython.display.html
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', quality=100)

## You can also just do this in Colab/Jupyter, some "magic":
%config InlineBackend.figure_format='retina'

In [ ]:
"""
Specify constants used across the rest of the notebook.
These have to do with the styling and an also consistency in the 
names of the fields or text labels used across the various code snippets.
"""

KAGGLE_PROCESSED_DATASET_PATH='../input/kaggle-machine-learning-data-science-survey-ext/preprocessed-kaggle-2017-to-2020/'

# References:
#   https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
#   https://docs.python.org/3/library/string.html#format-specification-mini-language for format styling

colour_map = sns.light_palette("#235f83", as_cmap=True)
DEFAULT_WHOLE_NUMBERS = "{:}"
DECIMAL_PLACES_FOR_PERCENTAGES = 2
AVERAGE_AGGREGATED_GROWTH = 'Average Aggregated Growth (%)'

NA_LABEL = 'N/A'
NOT_AVAILABLE = "Unknown / Not Specified"
COUNTRY_NOT_IN_THE_LIST = 'Other' # another form of NOT_AVAILABLE (the country is not on the list)
SURVEY_RESPONSE_COLUMN = 'Survey responses'

YEARS = [2017, 2018, 2019, 2020]
FIRST_YEAR = YEARS[0]
LAST_YEAR = YEARS[-1:][0]

ACTIVE_TEXT = 'Active on Kaggle' 
NOT_PRESENT_TEXT = 'Not Present on Kaggle' 
ACTIVE_COL = f'{ACTIVE_TEXT} count'
NOT_PRESENT_COL = f'{NOT_PRESENT_TEXT} count'


STYLES = [
    dict(selector="th", props=[("font-size", "120%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("font-size", "120%"),
                                    ("caption-side", "bottom")])
]

def plot_growth_line_chart(dataframe: pd.DataFrame, plot_title: str, x_axis_column: str, y_axis_column: str, 
                    y_axis_labels_column: str, text_for_inside_of_plot: str, decimal_places_for_labels: int = 0,
                    annotation_text_x = 0.60, annotation_text_y = 0.60,
                    arrow_adjust_x: float = .065, arrow_adjust_y: float = -.010,
                    plot_canvas = None):
    """
    Draw a line chart and plot the points from a dataframe, 
    specified by its x and y column names annotate it with 
    a text and also an arrow illustrating the direction of 
    change mid-way into the plot (graph).
    
    Additional (default values) parameters provided to adjust 
    the position of the annotated text and/or the arrowhead.
    """
    
    invoke_show_at_the_end = True
    if plot_canvas:
        dataframe.plot(ax=plot_canvas, x=x_axis_column, y=y_axis_column, color=['darkorange'], 
                           legend=False, marker='o', linestyle='--', markersize='10')
        invoke_show_at_the_end = False
    else:
        plot_canvas = dataframe.plot(x=x_axis_column, y=y_axis_column, color=['darkorange'], 
                         legend=False, marker='o', linestyle='--', markersize='10')

    plot_canvas.spines["top"].set_visible(False)
    plot_canvas.spines["right"].set_visible(False)
    plot_canvas.spines["left"].set_visible(False)

    plot_canvas.set_title(plot_title, y=1.5)

    y_axis = plot_canvas.axes.get_yaxis()
    y_axis.set_visible(False)

    for x, y in zip(range(len(YEARS)), dataframe[y_axis_labels_column]):
        label = "{:." + str(decimal_places_for_labels) + "f}"
        label = label.format(y)
        plot_canvas.annotate(label,
                       (x, y),
                       textcoords="offset points",
                       xytext=(0,20),
                       ha='center',
                       color='darkorange',
                       fontsize=14)
        """
        Remember the x and y position of 
        the previous reading to be able to use it from the next one
        """
        if (x == 1):
            prev_x = x
            prev_y = y

        """
        When we come to the point where the increase 
        in value(s) takes place we draw and arrow 
        from the previous point to this one
        """
        if (x == 2): # only for the 3rd data point
            plot_canvas.annotate('', 
                       xy=(x+.055, y-.005), xycoords='data',
                       xytext=(prev_x + arrow_adjust_x, prev_y + arrow_adjust_y), textcoords='data',                        
                       arrowprops=dict(facecolor='black', shrink=0.2),
                       horizontalalignment='right', verticalalignment='top')

    textstr = '\n'.join(('', text_for_inside_of_plot)) 
    plot_canvas.text(annotation_text_x, annotation_text_y, textstr, transform=plot_canvas.transAxes, 
                     fontsize=14, verticalalignment='top', c='#FF0547')

    if invoke_show_at_the_end:
        plt.tight_layout()
        plt.show()
    
def color_zero_or_negative_red(val):
    """
    Takes a scalar and returns a string with
    the CSS property `'color: red'` for negative 
    or zero valued strings, dark-gray otherwise.
    """
    color = 'red' if val <= 0 else 'dark-gray'
    return 'color: %s' % color

<a id='ToC'></a>

----------

# <div class="section_title">Table of contents

- [Introduction](#introduction)
- _**[Summary](#summary)**_
- _[Revisions](#revisions)_
- [Answering questions with the dataset](#questions)
    - [1. How many responses are we getting from the survey sent out each year?](#responses_stats)
       - [a. What is the contribution to the Kaggle community?](#responses_stats)
       - [b. How are the responses distributed across the continents, regions and countries across the years (on a map)?](#responses_on_world_map_by_year)
    - [2. Over the years how have the number of responses grown, or have they stayed the same or declined globally?]( #active_members_response)
      - [a. Survey response metrics from countries in the continent of Asia](#survey_metrics_asia)
      - [b. Survey response metrics from countries in the continent of Europe](#survey_metrics_europe)
      - [c. Survey response metrics from countries in the continent of Oceania](#survey_metrics_oceania)
      - [d. Survey response metrics from countries in the continent of Africa](#survey_metrics_africa)
      - [e. Survey response metrics from countries in the continent of North America](#survey_metrics_north_america)
      - [f. Survey response metrics from countries in the continent of South America](#survey_metrics_south_america)
      - [g. Summary of the survey response metrics of all the continents](#survey_metrics_all_continents)
    - [3. Which countries are growing in participations and which ones are retreating?](#growing_and_retreating)
      - [a. Countries growing in participation (rising stars)](#growing_in_participation)
      - [b. Countries retreating (mid-aged and veterans)](#retreating)
    - [4. Which countries are NOT yet on Kaggle's world map?](#active_or_not_on_kaggle)
      - [a. Countries active on Kaggle](#countries_active_on_kaggle)
      - [b. Breakdown of the number of countries active on Kaggle (by their Development Status)](#countries_active_on_kaggle_by_dev_status)
      - [c. Countries NOT present on Kaggle](#countries_not_present_on_kaggle)
      - [d. Breakdown of the number of countries NOT present on Kaggle (by their Development Status)](#countries_not_active_on_kaggle_by_dev_status)
      - [e. Map of countries that are NOT present or active on Kaggle (a wishlist)](#kaggle_wish_list_map)
      - [f. Comparing statistics between countries that are active on Kaggle and those that are NOT](#comparing_countries_kaggle_status)
      - [g. A walk-through the years with survey responses from the continents, regions and countries of the Kaggle world](#racing_charts)
- [Overall conclusion](#overallConclusion)
- [Credit and thanks](#credits)

<a id='introduction'></a>

----------

## <div class="subsection_title">Introduction

### <center>"Gathering Insights Through Changing Signals From Global and Local Participants"</center>

[![](https://storage.googleapis.com/kaggle-competitions/kaggle/23724/logos/header.png?t=2020-10-31-23-22-58)](https://www.kaggle.com/c/kaggle-survey-2020)
### [2020 Kaggle Machine Learning & Data Science Survey](https://www.kaggle.com/c/kaggle-survey-2020)

**_[The most comprehensive dataset available on the state of ML and data science](https://www.kaggle.com/c/kaggle-survey-2020)_**

---
### [State of Data Science and Machine Learning 2020](https://www.kaggle.com/kaggle-survey-2020)
### [State of Data Science and Machine Learning 2019](https://www.kaggle.com/kaggle-survey-2019)
### [Kaggle Survey 2018 Noteobook](https://www.kaggle.com/kaggle/kaggle-survey-2018/notebooks)  
### [Kaggle Survey 2017 Noteobook](https://www.kaggle.com/kaggle/kaggle-survey-2017/notebooks)  

The purpose or aim of this submission and notebook (including the accompanying notebook and datasets) are many folds:
    
- study and understand the Kaggle Data Science / Machine Learning Survey dataset (and also other related datasets), everyone else's notebooks and visualisation techniques
- highlight insights about the dataset that may be lightly different from the others shown so far (I hope all of them are useful and help learn more about everything these surveys are about)
- to touch upon NOT just community growth but ALSO "sustainability and support"
- the intention is to use simple ideas and Data Science and Exploratory Data Analysis methods and present them in this notebook (and supporting notebooks and datasets), so they are easier to read and reason about
- also highlight that there were pain-points when assembling the datasets (thanks to another fellow Kaggler's contributions that helped the situation)
- to share suggestions and ideas for improvements and other things that you may see in the course of this notebook

I hope that the feedback and insights provided here will help with the existing and future process of gathering feedback via such annual surveys. Also to highlight that the title of the notebook, is trying to convey a few hints (or messages):
    
- the outreach of Kaggle as a community and what the maintainers ARE doing to reach out, and support
- the outreach of Kaggle as a community and what the maintainers COULD be doing to sustain and support the existing Data Science / Machine Learning community
- also what could be done to include those in the wing who are not participating but can be reached out

As in my view, survey processes and the responses received from them act as a pulse about the community at different points in time - they are a great way to do community-health checks. Because the quality of the data gathered from such an action will depend on this factor and will also be reflected throughout the process and in the data collected. In our case, the metrics are going to be super helpful to throw light on such and other insights.

The views and opinions I'm making are solely based on a constrained framework and the condition that the data we have collected may be reflecting only a portion of the Data Science and Machine Learning world out there (the little we know from what we receive as responses). And it is down to us to further prove or disprove these arguments and conjectures. So the analysis and conclusions may have limitations outside the context of this framework or condition. I'm also indirectly eluding to presence of bias in the results/outcome/interpretations from the data, which we will see through the following sections of the report (the different angles and lenses drawn out in the rest of the notebook).

I agree that this approach is a *bit meta* as we are using the survey metrics (not just the data gathered by the survey) to make analysis, assessments and conclusions about the survey itself and how this can be changed or improved at the same time assessing the current community and the future participants of the community.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='summary'></a>

----------
> ## <div class="subsection_title">Summary
>
> - We sifted through many survey responses datasets, preprocessed them and merged them to create a combined dataset and further combined it with other datasets to enrich the existing information gathered by the survey activities.
> - Several countries across the six continents of **Asia**, **Africa**, **Europe**, **the Americas** and **Oceania (Australia and New Zealand)** participated in providing feedback since **2017** through to **2020**.
> - Looking at the metrics from the Survey responses shows how vibrant the community is and how they have spent time and have been engaged for a long time learning and contributing back. Further they show that there are new comers who are rising fast and the veterans and mid-aged are slowing down and these are more than a handful of countries in the two or three groups. You can read all about them here <a href=#growing_and_retreating>3. Which countries are growing in participations and which ones are retreating?</a>.
> - A hint or highlight that these regions of the world may need a bit more attention to sustain and support their presence and participation, you can see them in <a href='#active_members_response'>2. Over the years how has the number of responses grown, or have they stayed the same or declined globally?</a>. You will see countries in the **African** and **Middle-Eastern** regions are generally doing well, closely followed by **Asia** and **South America**. These all fall under the **Developed** and **Developed/Developing** categories.
> - Unfortunately or maybe due to economic cycles, participations from the **Developed** countries across the board are not on a growth trajectory and as we know a lot of exchanges take place between the **Developed** countries and rest of the world so it would be good to find out the reason(s) behind the decline in the participation. These might just be early signals leading to something else.
> - Many countries that have NOT been represented in the survey responses since the beginning, even though some of them might have participated in the Kaggle forums in some form or shape (may not as a registered member). And it would be good to reach out to them to find out the reason(s) for their absence. Maybe they don't know about Kaggle or there isn't enough support or incentives for them to come forward. You can find these countries mentioned and listed in the <a href='#active_or_not_on_kaggle'>3. Which countries are NOT yet on Kaggle's world map?</a> section down below in the notebook.
> - We may almost have no contact or feedback or signals from the **Less/Under Developed** countries contributing to the surveys -- I haven't looked into this in detail so I could be wrong about it but different aspects of the analysis show it nevertheless. I have highlighted this in these two subsections <a href=#countries_not_active_on_kaggle_by_dev_status>d. Breakdown of the number of countries NOT present on Kaggle (by their Development Status)</a> and <a href='#comparing_countries_kaggle_status'>e. Comparing statistics between countries that are active on Kaggle and those that are NOT</a>.
> - More specifically we could look at these sections and subsections if we would like to think about an action-plan <a href=#growing_and_retreating>3. Which countries are growing in participations and which ones are retreating?</a> (subsections <a href=#growing_in_participation>a. Countries growing in participation (rising stars)</a> and
<a href=#retreating>b. Countries retreating (mid-aged and veterans)</a>).
> - For brevity reasons, not all of the data from the Kaggle survey data or the other accumulated dataset could be used but given purpose and encourage I'm happy explore them further at a later time.
> - Pain points were observed when trying to process the survey datasets and also preprocessing other external datasets.
> - Notebooks to create external datasets and links to other additional datasets have been provided, and these can be reused not just for this competition but also for any other related purposes as well. Especially those involving cities, countries, continents and regions of the world.
> - The bulk of the analysis can be found in the next section <a href=#questions>Answering questions with the dataset</a>. Any changes due to updates or corrections can be seen via the [Revisions section](#revisions).
> - And to learn about the closing discussions, please also review the <a href=#overallConclusion>Overall conclusion</a> and [Credit and thanks](#credits) sections -- as always the <a href='#ToC'>Table of Contents</a> can help navigate through this notebook with ease.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='revisions'></a>

----------
> ## <div class="subsection_title">Revisions

During the course of the preparation of the [Kaggle datasets (from 2017 to 2020)](https://www.kaggle.com/neomatrix369/kaggle-machine-learning-data-science-survey-ext) and also [another dataset](https://www.kaggle.com/neomatrix369/world-bank-data-1960-to-2016-extended) there were discrepancies caused due to the presence of whitespaces and other forms of duplications in the data that through various checks were finally ironed out (can be seen in this notebook [Kaggle Machine Learning & Data Science (data-prep)](https://www.kaggle.com/neomatrix369/kaggle-machine-learning-data-science-data-prep/)).

This has lead to multiple revisions but the last one ([version 26](https://www.kaggle.com/neomatrix369/kaggle-global-outreach-analysis?scriptVersionId=51423441)) is now overridden by this update and any future ones made.


You will see the tables in the following sections changed:

- [a. Survey response metrics from countries in the continent of Asia](#survey_metrics_asia)
- [b. Survey response metrics from countries in the continent of Europe](#survey_metrics_europe)
- [c. Survey response metrics from countries in the continent of Oceania](#survey_metrics_oceania)
- [d. Survey response metrics from countries in the continent of Africa](#survey_metrics_africa)
- [e. Survey response metrics from countries in the continent of North America](#survey_metrics_north_america)
- [f. Survey response metrics from countries in the continent of South America](#survey_metrics_south_america)
- and all other dependent tables and graphs that draw information or make conclusions based on the data from the above changed sections

As already mentioned and explained in the [Overall conclusion](#overallConclusion) section that if a change of this nature was to come about, it wouldn't change the long-term or even medium-term outlook (the big-picture still remains intact). Although we could see a handful of countries (at worst a continent or two) change positions. These changes can be seen when comparing the outputs of this version with that of [version 26](https://www.kaggle.com/neomatrix369/kaggle-global-outreach-analysis?scriptVersionId=51423441). In short, there have been changes of a few (+/-) percentage points across a handful of countries, which have impacted metrics both at region and continent levels only mildly, but the overall big-picture conclusions remain the same.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='questions'></a>

----------

## <div class="subsection_title">Answering questions with the dataset 

<a id='responses_stats'></a>

----------

### <div class="subsubsection_title"> 1. How many responses are we getting from the survey sent out each year?

### <div class="subsubsection_title"> a. What is the contribution to the Kaggle community?

#### Here we are trying to suggest a number of things:

- are all the registered members on Kaggle receiving the survey requests each year?
- are they taking the time to fill in and return?
- from those who are doing so how many are they?
- is this proportional to the number of registered members (increasing or decreasing each year)?
- does this make the sample data representative of the Data Science / Machine Learning community worldwide?

In [ ]:
"""
Load our survey responses stats table and display them after 
performing some amendments and prettifying the dataframe to be displayed
"""
survey_response_stats = pd.read_csv(f'{KAGGLE_PROCESSED_DATASET_PATH}/survey_response_stats.csv')
survey_response_stats = survey_response_stats.rename(columns={'Year': 'Survey Year',
                                                              'Total_responses_count': 'Total Survey Responses',
                                                              'Total_members_count': 'Total Kaggle Members',
                                                              'Members_to_response_ratio': 'Members / Responses ratio',
                                                              'Total_responses_count_pct_change': 'Total Survey Responses increase / decrease',
                                                              'Members_to_response_ratio_pct_change': 'Members / Responses increase / decrease',
                                                              'Total_members_count_pct_change': 'Total Kaggle Members increase / decrease'})
colums_to_display = ['Total Survey Responses','Total Kaggle Members', 'Members / Responses ratio', 
                     'Total Survey Responses increase / decrease']
survey_response_stats['Survey Year'] = survey_response_stats['Survey Year'].astype(str)
survey_response_stats = survey_response_stats.set_index('Survey Year')
survey_response_stats = survey_response_stats.fillna(0)
format_numeric_columns = {
    "Total Survey Responses": "{:.0f}", "Total Kaggle Members": "{:.0f}", 
     "Members / Responses ratio": "{:.4f}", "Total Survey Responses increase / decrease": "{:.4f}"
}
survey_response_stats[colums_to_display].style.format(format_numeric_columns, na_rep=NA_LABEL).set_table_styles(STYLES) \
               .bar(subset=['Total Survey Responses', 'Members / Responses ratio', 'Total Survey Responses increase / decrease'],
                            align='mid', color=['#d65f5f', '#64c2ed'])

In [ ]:
"""
We need to do this as otherwise we can't use 
the year to be displayed on the x-axis.
""" 
if survey_response_stats.index.name == 'Survey Year':
    survey_response_stats = survey_response_stats.reset_index()

plt.rcParams['figure.figsize'] = [15.0, 5.0]
plot_growth_line_chart(survey_response_stats, "Growth in Kaggle membership", 'Survey Year', 
                'Total Kaggle Members', 'Total Kaggle Members','Rapid increase in membership',
               arrow_adjust_x=0.20, arrow_adjust_y=-0.20,
               annotation_text_x = 0.55, annotation_text_y = 0.50)

In [ ]:
plt.rcParams['figure.figsize'] = [15.0, 5.0]
plot_growth_line_chart(survey_response_stats, "Growth in Survey Responses from Kaggle members (percentages)", 
                'Survey Year', 'Total Survey Responses increase / decrease', 'Members / Responses increase / decrease', 
                'Decline in the number of Survey responses', decimal_places_for_labels=6,
                 annotation_text_x = 0.525, annotation_text_y = 0.75)

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">We can see from the two metrics (from the two lineplots) that although each year Kaggle's membership (community) **increases** in numbers, the number of responses received for surveys sent out each year is **on a decline**. Also the proportion of Kaggle members responding to the survey compared to the total number of registered Kaggle members is a significantly low figure (a very big difference). For each year these numbers are **0.2803%**, **0.3997%**, **0.3302%** and **0.3343%** for the years **2017**, **2018**, **2019**, and **2020** respectively. There could be many reasons for this and it would be good to investigate this as it's a recurring event. Another question that comes up immediately with regards to the above, since the ratios are such small, can we say that the sample is representative of the Data Science/Machine Learning community at large or even on Kaggle, this remains to be seen. And maybe for myself or others to provide some more reasoning or reports on this. Besides, it would be good to compare these numbers against that of active Kaggle members (for each year) and then try to conclude from them.

In [ ]:
kaggle_combined_worldwide_df = pd.read_csv(f'{KAGGLE_PROCESSED_DATASET_PATH}/kaggle_2017_to_2020_and_countries.csv')
kaggle_combined_worldwide_df[SURVEY_RESPONSE_COLUMN] = 1

In [ ]:
other_unknown_or_not_specified_filter = (kaggle_combined_worldwide_df['Country'] == NOT_AVAILABLE) | (kaggle_combined_worldwide_df['Country'] == COUNTRY_NOT_IN_THE_LIST)
seleted_columns = ['Year', 'Country', 'latitude', 'longitude']
countries_with_their_locations = kaggle_combined_worldwide_df[~other_unknown_or_not_specified_filter].copy()

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='responses_on_world_map_by_year'></a>

----------

### <div class="subsubsection_title"> b. How are the responses distributed across the continents, regions and countries across the years (on a map)? 

In [ ]:
def create_popup(country_name: str, responses: list) -> str:
    country_name = country_name.replace(" ", '<font color="white">:</font>')
    output = f'<i><strong>{country_name}</strong></i><br><br>'
    for year, response_count in zip(YEARS, responses):
        if response_count == 0: 
            response_count = 'N/A'
        else:
            response_count = int(response_count)
        output = f'''{output}<strong>{year}</strong>:<font color="white">:</font>{response_count}<br/>'''
    return output


responses_per_country_by_year = countries_with_their_locations.pivot_table(SURVEY_RESPONSE_COLUMN, 
                                                                           index=['Country','latitude', 'longitude'], columns='Year',
                                                                          aggfunc='count')
responses_per_country_by_year = responses_per_country_by_year.fillna(0).reset_index()


survey_responses_map = folium.Map(
    location=[0, 0], 
    tiles="stamenterrain",
    zoom_start=1
)

locations_data = list(zip(countries_with_their_locations['latitude'].values, countries_with_their_locations['longitude'].values))

for _, row in responses_per_country_by_year.iterrows():
    folium.CircleMarker(
            location=[row['latitude'] + 0.5, row['longitude'] + 0.5],
            popup=create_popup(row['Country'], row[YEARS]),
            fill=True,
            color="#007849",
            radius= 1.25 * np.log(np.sum(row[YEARS])),
            label=f"{np.sum(row[YEARS])}",
        ).add_to(survey_responses_map)

FastMarkerCluster(data=countries_with_their_locations[['latitude', 'longitude']]).add_to(survey_responses_map)

folium.LayerControl().add_to(survey_responses_map)

display(survey_responses_map)

<i><p style="font-size:16px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">Nothing like an interactive map that shows the number of responses (by the size of the circles and also the yellow markers) across the continents, regions and countries. You can also zoom into the regions/continents to find out which countires have been actively participating over one or more years and clicking on the circles reveals individual years of responses for that country.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

In [ ]:
def calculate_aggregated_growth_rate(dataframe: pd.DataFrame) -> list:
    """
    Using the response data for all the years in the pivot table (dataset),
    calculate the aggregated growth rate across the years into a 
    new column called "Aggregated Growth (%)".
    
    Ignore columns with nan values and if there isn't enough data 
    return nan (which will be handled and formatted by the other 
    aspects of the flow).
    """    
    results = []
    for each_row in dataframe.iterrows():
        if each_row[0][0] == '_Total':
            results.append(0)
        else:
            year_info = each_row[1][:'Year'][SURVEY_RESPONSE_COLUMN]
            growth_per_year = []
            for each_year in YEARS[1:]:
                prev_year = each_year - 1
                if year_info[prev_year] != 0:
                    if (not np.isnan(year_info[prev_year])) and (not np.isnan(year_info[each_year])):
                        growth = 100 * ((year_info[each_year] / year_info[prev_year]) - 1)
                        growth_per_year.append(growth)

            mean_growth = round(np.mean(growth_per_year), DECIMAL_PLACES_FOR_PERCENTAGES)
            if len(growth_per_year) == 0:
                mean_growth = np.nan
            results.append(mean_growth)
    return results

def get_summary_stats_for(continent: str) -> pd.DataFrame:
    """
    From the pre-computed pivot tables for each continent (originally a single table),
    fetch summary statistics for each continent and return a new dataframe with 
    those details in it.
    
    The new dataframe will not have the "Total" column but will contain the responses 
    for each of the years and the "Average Aggregated Growth" rate of all the responses.
    """       
    each_continent_pivot = kaggle_combined_pivot_by_continent[continent]
    for each_row in each_continent_pivot.iterrows():
        if each_row[0][0] == 'Total':
            year_info = each_row[1][:'Year'][SURVEY_RESPONSE_COLUMN]
            overall_growth = each_row[1][:'Year'][AVERAGE_AGGREGATED_GROWTH][0]
            dataframe = pd.DataFrame(year_info)

            dataframe = dataframe.T.reset_index(drop=True)
            dataframe[AVERAGE_AGGREGATED_GROWTH] = overall_growth
            dataframe.columns.name = ''
            dataframe['Continent'] = continent
            return dataframe.drop(columns=['Total'])

def sort_values_with_total_at_tail_end(dataframe: pd.DataFrame, by_col, ascending=False) -> pd.DataFrame:
    body = dataframe[:-1].sort_values(by=by_col, ascending=ascending)
    footer = dataframe[-1:]
    
    return pd.concat([body, footer])

In [ ]:
## Pivot table reference: https://datagy.io/python-pivot-tables/#plot-pivot-tables

"""
Split the bigger pivot table into smaller ones so 
they can be processed further much easier.

Hold them in a dictionary.

Do the same for the summary stats for each of the continent, 
intially hold them in dictionary to help concatenating them later on.
"""

continents = list(kaggle_combined_worldwide_df['Continent'].value_counts().to_dict().keys())

kaggle_combined_pivot_by_continent = {}
kaggle_combined_pivot_by_region = pd.DataFrame()
kaggle_pivot_by_continent_summary_stats = {}

for each_continent in continents:
    apply_filter = kaggle_combined_worldwide_df['Continent'] == each_continent
    filtered_kaggle_combined_worldwide_df = kaggle_combined_worldwide_df[apply_filter]
    kaggle_combined_pivot_by_continent[each_continent] = filtered_kaggle_combined_worldwide_df.pivot_table(values = [SURVEY_RESPONSE_COLUMN], 
                                         index=['Region', 'Country'], columns='Year', 
                                                               aggfunc='sum', margins=True, margins_name='Total').copy()
    each_region = filtered_kaggle_combined_worldwide_df.pivot_table(values = [SURVEY_RESPONSE_COLUMN],
                                         index=['Region'], columns='Year', aggfunc='sum').copy()
    kaggle_combined_pivot_by_region = pd.concat([kaggle_combined_pivot_by_region, each_region])

    kaggle_combined_pivot_by_continent[each_continent][AVERAGE_AGGREGATED_GROWTH] = \
                              calculate_aggregated_growth_rate(kaggle_combined_pivot_by_continent[each_continent])
    
    kaggle_combined_pivot_by_continent[each_continent] = \
                            sort_values_with_total_at_tail_end(kaggle_combined_pivot_by_continent[each_continent], AVERAGE_AGGREGATED_GROWTH)
    
    kaggle_pivot_by_continent_summary_stats[each_continent] = get_summary_stats_for(each_continent)
    
"""
From the pre-computed pivot tables for each continent (originally a single table),
fetch summary statistics for each continent and return a new dataframe with 
those details in it.
"""
kaggle_pivot_all_continents_summary_stats = pd.DataFrame()
for each_dataframe in kaggle_pivot_by_continent_summary_stats:
    kaggle_pivot_all_continents_summary_stats = pd.concat([kaggle_pivot_all_continents_summary_stats, 
                                                           kaggle_pivot_by_continent_summary_stats[each_dataframe]])
kaggle_pivot_all_continents_summary_stats = kaggle_pivot_all_continents_summary_stats.reset_index(drop=True)

columns = ['Continent'] + YEARS + [AVERAGE_AGGREGATED_GROWTH]
kaggle_pivot_all_continents_summary_stats = kaggle_pivot_all_continents_summary_stats[columns].set_index('Continent')

aggregates = kaggle_pivot_all_continents_summary_stats.agg(['sum'])
aggregates[AVERAGE_AGGREGATED_GROWTH] = aggregates[AVERAGE_AGGREGATED_GROWTH] / kaggle_pivot_all_continents_summary_stats.shape[0]
aggregates[AVERAGE_AGGREGATED_GROWTH] = round(aggregates[AVERAGE_AGGREGATED_GROWTH], DECIMAL_PLACES_FOR_PERCENTAGES)
aggregates = aggregates.reset_index(drop=True)
aggregates['Continent'] = 'Total'
aggregates = aggregates.set_index('Continent')

kaggle_pivot_all_continents_summary_stats = \
                        sort_values_with_total_at_tail_end(kaggle_pivot_all_continents_summary_stats, AVERAGE_AGGREGATED_GROWTH)

kaggle_pivot_all_continents_summary_stats = pd.concat([kaggle_pivot_all_continents_summary_stats, aggregates])
kaggle_combined_pivot_by_region = kaggle_combined_pivot_by_region.reset_index()

In [ ]:
def prettify_display(dataframe: pd.DataFrame, 
                     caption: str = '[No caption has been set]', 
                     custom_format = DEFAULT_WHOLE_NUMBERS):
    """
    Prettify the way the dataframe will be displayed in the 
    notebook using the Styling (Styles) features of Pandas.
    It will use the dataframe containing the summary stats 
    of all the continents
    
    The constants needed for the purpose are provided in the body.
    """
    return dataframe.style \
                 .format(custom_format, na_rep=NA_LABEL) \
                 .set_caption(caption) \
                 .set_table_styles(STYLES) \
                 .background_gradient(cmap=colour_map) \
                 .applymap(color_zero_or_negative_red) # maintain the order of the functions called 
                                                       # or else the colours are not set correctly

def get_formatted_table_for_all_continents() -> pd.DataFrame: 
    """
    Return a prettified dataframe containing summary stats of all the continents
    """ 
    return prettify_display(kaggle_pivot_all_continents_summary_stats, 
                            'Summary of all metrics of the survey responses from all the continents.')

def get_responses_stats_for_continent(name: str) -> pd.DataFrame: 
    """
    Return a prettified dataframe containing stats for each of the continents.
    
    Prettify the way the dataframe will be displayed in the 
    notebook using the Styling (Styles) features of Pandas.
    It will use the dictionary containing the responses stats 
    for each of the continents and can be called out by passing 
    the correct name of the continent.
    
    A footnote also explains the meaning of "N/A" when displayed 
    under different columns.
    """
    return prettify_display(kaggle_combined_pivot_by_continent[name],
                           f'Summary all metrics of the survey responses from each country in the continent of "{name}". \n' \
                           f'Note if "{NA_LABEL}" is present under the "Response" columns mean we do not have data for that year. \n' \
                           f'Note "{NA_LABEL}" under the "{AVERAGE_AGGREGATED_GROWTH}" column means we could not compute it due too little or no data. \n' \
                           'The country or countries concerned might be new to the process or have submitted a survey only once in all these years.')

<a id='active_members_response'></a>

----------


### <div class="subsubsection_title"> 2. Over the years how have the number of responses grown, or have they stayed the same or declined globally?

### This is a much broader question and spans across the different regions and continents that Kaggle members come from. And here we are interested in finding out how have the number responses from the different countries been, what are the trends? We are seeking to find atleast three of these categories increase, decrease, and no change.

<a id='survey_metrics_asia'></a>

----------

#### <div class="subsubsubsection_title">*a. Survey response metrics from countries in the continent of Asia*

In [ ]:
get_responses_stats_for_continent('Asia')

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">From the above table, we can see that a couple of countries in the continent of **Asia** (with the breakdown of the different **Regions**) have recently joined the party. While another handful have only completed surveys once or twice but not all the four years. Therefore those countries do not show a growth in their participation. Another pocketful of countries show a single digit growth even though they have been regularly completing the surveys. This sudden drop could be attributed to situations in those countries impacting the Data Science / Machine Learning communities there (it's something to look closer into) -- and it could be outside their control as personal or learning (educational) priorities change if economic or political challenges approach a region. Two other countries i.e. **Taiwan** and **Singapore** are showing a negative growth and this stands out in the table. Despite these ups and downs the region is managing well, as these losses are offset by other strong participation from **India**, **China**, **Japan**, **Saudi Arabia**, **Turkey** and a handful of others. A small exception to note aout **Saudi Arabia** as it's participation is only since a couple of years - a new member as compared to many others in this region of the world. Another country to note is **China**, it started off with a big bang of **~200%** growth but last few years it's been on a decline, but it's initial start has helped maintain the average growth rate - I do agree this isn't the real picture though but statistical measures like average can hide trend many a times.
    
<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**EDIT:** As mentioned in the [Revisions](#revisions) section, due to changes in the dataset, the following countries have moved up or down the list as compared to the previous versions of this report: **Saudi Arabia**, **China**, **India**, and **Turkey**. This may have also changed the relative positions of other countries in the table. And the overall growth rate of the region has been corrected from **37.27%** to **27.67%**.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='survey_metrics_europe'></a>

----------

#### <div class="subsubsubsection_title">*b. Survey response metrics from countries in the continent of Europe*

In [ ]:
get_responses_stats_for_continent('Europe')

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">From the above table, we can see that all the countries in the continent of **Europe** (with the breakdown of the different **Regions**) have participated since when surveys were being collected. While a handful of the countries have not submitted surveys at each time in the past years. Therefore those countries show a shunted or decline in their growth in their participation. Another handful of countries show a **single digit** and even **negative** growth when it comes to completing and submitting the surveys, which is heavily impacting the overall growth of the region (a growth of **-0.79%** (negative growth) is something to think about and investigate further). A **double-digit negative growth** from a few countries is greatly impacting the growth in participation from this region. There are certainly good reasons for this to happen, and this could be investigated further.
    
<i><p style="font-size:19px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**C.W. OF IND. STATES** stands for <a href=https://en.wikipedia.org/wiki/Commonwealth_of_Independent_States>Commonwealth of Independent States</a>  and Ukraine is one of the countries among a handful of them.
    
<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**EDIT:** As mentioned in the [Revisions](#revisions) section, due to changes in the dataset, the following countries have moved up or down the list as compared to the previous versions of this report: **Czech Republic**, **Sweden**, **Russia**, **Spain** and **Hungary**. This may have also changed the relative positions of other countries in the table as a result. And the overall growth rate of the region has been corrected from **0.92%** to **-0.79%**.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='survey_metrics_oceania'></a>

----------

#### <div class="subsubsubsection_title">*c. Survey response metrics from countries in the continent of Oceania*

In [ ]:
get_responses_stats_for_continent('Oceania')

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">From the above table we see the two countries in the continent of **Oceania** (also known as **Australiasia** sometimes) have participated since the beginning of the time when surveys were collected. We immediately also notice that **New Zealand** has missed submissions in **2020** (and this could well be due the recent challenges all nations have been facing since a year now). Although they also have negative growth when it comes to completing the surveys as we can see there is a year-on-year decline in their participation since the beginning and the decline is a significant **14.86%** (i.e. negative growth of **14.86%**). Similarly, **Australia**'s participation is declining by **18.08%**. Both of which brings the net participation from this region down to **-22.32%**.
    
    
<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**EDIT:** As mentioned in the [Revisions](#revisions) section, due to changes in the dataset, the table above has been impacted. We can see numbers of both the countries have changed a bit. This may have also changed the relative positions of other countries in the table as a result. And the overall growth rate of the region has been corrected from **-20.74%** to **-22.32%**.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='survey_metrics_africa'></a>

----------

#### <div class="subsubsubsection_title">*d. Survey response metrics from countries in the continent of Africa*

In [ ]:
get_responses_stats_for_continent('Africa')

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">From the above table, we can see the various countries in the continent of **Africa** (with the breakdown of the different **Regions**) although many of them have not participated since the beginning of the time when surveys were collected. We have two recent joiners **Algeria** and **Ghana** (**Algeria** missed out a submission in **2020**). The overall growth from this region is promising and highly positive across all the years so far. Of all of them, **Nigeria** has a staggering growth of **98.45%** - this shows they take such things very seriously, and Kaggle is a serious subject in this region. The overall growth in each of the countries and the continent as a whole is a very impressive **59.89%** since the start of their participations.
    
<i><p style="font-size:19px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**SUB-SAHARAN AFRICA** stands for <a href=https://en.wikipedia.org/wiki/Sub-Saharan_Africa>Sub-Saharan Africa</a>.


<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**EDIT:** As mentioned in the [Revisions](#revisions) section, due to changes in the dataset, the table above has been impacted. We can see numbers of many of the countries have changed a bit. But the relative positions of NONE of the  countries in the table have changed as a result. Although the overall growth rate of the region has been corrected from **60.33%** to **59.89%**, which is still a big positive growth.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='survey_metrics_north_america'></a>

----------

#### <div class="subsubsubsection_title">*e. Survey response metrics from countries in the continent of North America*

In [ ]:
get_responses_stats_for_continent('North America')

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">From the above table, we can see the three countries in the continent of **North America** (with the breakdown of the two **Regions**) - they have participated since the beginning of the time when surveys were collected. Although they have accrued negative growth in their participation as there is a decline in their participation since the beginning, except for **Mexico** which leads in this space with a **24.42%** growth. We cannot not notice that the **United States** constituties to a large chunk of the submissions not just in this continent but globally as well, although the numbers are declining over the years as can be seen from a **-16.57%** growth.
    
<i><p style="font-size:19px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**LATIN AMER. & CARIB** stands for <a href=https://en.wikipedia.org/wiki/Latin_America_and_the_Caribbean>Latin America and the Caribbean</a>.

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**EDIT:** As mentioned in the [Revisions](#revisions) section, due to changes in the dataset, the table above has been impacted. We can see numbers of the three countries have changed accordingly, although their relative positions in the table have not changed. And the overall growth rate of the region has been corrected from **-9.23%** to **-14.13%**.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='survey_metrics_south_america'></a>

----------

#### <div class="subsubsubsection_title">*f. Survey response metrics from countries in the continent of South America*

In [ ]:
get_responses_stats_for_continent('South America')

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">From the above table, we can see that almost all of the countries in the continent of **South America** who have participated since the beginning of the time when surveys were collected, except for **Peru**, that missed out in **2017** . And pleasantly so they all have a positive growth when it comes to completing the surveys as we can see there is a year-on-year an increase in their participation. A small observation being **Brazil** and **Chile** that have had their ups and downs but they lead in overall growth in this region. The overall growth in the continent is also higher than most others, although only after **Asia** and **Africa**.

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**EDIT:** As mentioned in the [Revisions](#revisions) section, due to changes in the dataset, the table above has been impacted mildly. We can see only **Brazil**'s numbers have changed accordingly, although their relative positions in the table have NOT changed. And the overall growth rate of the region has been corrected from **22.38%** to **20.69%**.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='survey_metrics_all_continents'></a>

----------

#### <div class="subsubsubsection_title">*g. Summary of the survey response metrics of all the continents* 

In [ ]:
get_formatted_table_for_all_continents()

In [ ]:
"""
Plot the individual growth (line) charts for each of the continents
Annotate their growth, decline or side-ways movements on the chart.
"""

known_continents = continents.copy() 
if 'Unknown / Not Specified' in known_continents:
    known_continents.remove('Unknown / Not Specified')

max_rows = (len(known_continents) // 2)
max_cols = 2
_, plots = plt.subplots(max_rows, max_cols, figsize=(15, 12.5))

arrow_head_text = [
    'steady growth after a good start', 'steep decline after a good start', 'steep decline after a good start',
    'slow climb after a good start', 'rapid growth from the start', 'rapid decline from the start',
]

annotation_text_coords = [(0.450, 0.75), (0.500, 1.10),
                          (0.525, 1.10), (0.525, 1),
                          (0.525, 0.60), (0.055, .65)]

for index, continent in enumerate(known_continents):
    row = index // 2
    col = index % 2
    each_continent_transformed_dataframe = kaggle_pivot_all_continents_summary_stats[continent:continent].reset_index().copy()
    each_continent_transformed_dataframe = each_continent_transformed_dataframe.drop(
        columns=['Continent', AVERAGE_AGGREGATED_GROWTH], axis=1).T.reset_index()
    each_continent_transformed_dataframe.columns=['Year', SURVEY_RESPONSE_COLUMN]
    each_continent_transformed_dataframe['Year'] = each_continent_transformed_dataframe['Year'].astype(str)
    each_continent_transformed_dataframe[SURVEY_RESPONSE_COLUMN] = each_continent_transformed_dataframe[SURVEY_RESPONSE_COLUMN].astype(int)
    each_continent_transformed_dataframe['Growth Rate'] = ((each_continent_transformed_dataframe[SURVEY_RESPONSE_COLUMN] / each_continent_transformed_dataframe[SURVEY_RESPONSE_COLUMN].shift(1)) - 1)
    each_continent_transformed_dataframe = each_continent_transformed_dataframe.fillna(0.0)
    annotation_text_x, annotation_text_y = annotation_text_coords[index]
    plot_growth_line_chart(each_continent_transformed_dataframe, 
                    f"Growth in '{continent}'", 
                    'Year', SURVEY_RESPONSE_COLUMN, SURVEY_RESPONSE_COLUMN, 
                    arrow_head_text[index], annotation_text_x = annotation_text_x, annotation_text_y = annotation_text_y,
                    decimal_places_for_labels=0, plot_canvas = plots[row][col])

plt.tight_layout()
plt.show()

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">Now looking at the summary stats of all of the different continents in one table is a lot exciting (including the line charts with their growths plotted). We can see the winners from the laggards to the decliners, clearly and all in one view. And as mentioned in the individual sub-sections for each of the continents above -- **Africa**, **Asia** and **South America** are on the rise while the other continents are declining or staggering behind. This is impacting the global growth in participations which is currently at **~11%** and declining slowly. I suggest some actions be taken to check why this is happening in each of the regions and/or how it can be fixed. We can say that the survey notebooks and reports  <a href="https://www.kaggle.com/kaggle/kaggle-survey-2017/notebooks">Kaggle Survey 2017 Noteobook</a> | <a href="https://www.kaggle.com/kaggle/kaggle-survey-2018/notebooks">Kaggle Survey 2018 Noteobook</a> | <a href="https://www.kaggle.com/kaggle-survey-2019">State of Data Science and Machine Learning 2019</a>  |  <a href="https://www.kaggle.com/kaggle-survey-2020">State of Data Science and Machine Learning 2020</a> from the years **2017**, **2018**, **2019** and **2020** respectively, have been influenced (or contributed by) these countries and regions of the world. As we can also see, as our world progresses we have to take into account and respect for those who wish to remain anonymous, and hence  the **Unknown / Not Specified** category, which is doing pretty fine with a **10.17%** growth (they have had their ups and downs).

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**EDIT:** As mentioned in the [Revisions](#revisions) section, due to changes in the dataset, the table above has been impacted accordingly, although the relative positions of the continents in the table have NOT changed but the overall growth rate of all the continents  put together has be corrected from **15.27%** to **11.6%**.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='growing_and_retreating'></a>

----------

### <div class="subsubsection_title"> 3. Which countries are growing in participations and which ones are retreating?

<a id='growing_in_participation'></a>

----------

#### <div class="subsubsubsection_title">*a. Countries growing in participation (rising stars)* 

In [ ]:
kaggle_all_countries_sorted_by_aag = pd.DataFrame()
for each_continent in continents:
    kaggle_all_countries_sorted_by_aag = pd.concat(
        [kaggle_all_countries_sorted_by_aag, kaggle_combined_pivot_by_continent[each_continent][:-1]]
    )

kaggle_all_countries_sorted_by_aag[AVERAGE_AGGREGATED_GROWTH] = kaggle_all_countries_sorted_by_aag[AVERAGE_AGGREGATED_GROWTH].fillna(0)

cutoff_filter = kaggle_all_countries_sorted_by_aag[AVERAGE_AGGREGATED_GROWTH] >= 10
kaggle_all_countries_sorted_by_aag_risers = kaggle_all_countries_sorted_by_aag[cutoff_filter].sort_values(by=AVERAGE_AGGREGATED_GROWTH, ascending=False)
kaggle_all_countries_sorted_by_aag_retreating = kaggle_all_countries_sorted_by_aag[~cutoff_filter].sort_values(by=AVERAGE_AGGREGATED_GROWTH, ascending=True) 

In [ ]:
prettify_display(kaggle_all_countries_sorted_by_aag_risers,
                 f'{kaggle_all_countries_sorted_by_aag_risers.shape[0]} out of ' \
                 f'{kaggle_combined_worldwide_df.Country.value_counts().shape[0]} countries from all across the world')

<i><p style="font-size:16px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">These are the increasingly participating countries to the Surveys Kaggle sends out each year, that we have been eluding to in the past sections. And I'm thinking their participation on other aspects of the Kaggle forum also reflect similar if not better participations. With the esception to the **Unknown / Not Specified** category that does not give a clear idea of a locations (they either did not share or did not wish to share it with the others). But there is was a sudden increase in this category and then they disappeared in **2019** and **2020**. I can already see a number of countries that have freshly graduated into the **Developed/Developing** countries list, sadly the number of **Developed** countries are on a decline and are falling out of this list into the other list (see section below <a href=#retreating>b. Countries retreating (mid-aged and veterans)</a>).

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**EDIT:** As mentioned in the [Revisions](#revisions) section, due to changes in the dataset, the table above has been impacted accordingly, and so has the relative positions of the countries in it has changed as mentioned in the above sections specific to each continent.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='retreating'></a>

----------

#### <div class="subsubsubsection_title">*b. Countries retreating (mid-aged and veterans)* 

In [ ]:
prettify_display(kaggle_all_countries_sorted_by_aag_retreating, 
                 f'{kaggle_all_countries_sorted_by_aag_retreating.shape[0]} out of ' \
                 f'{kaggle_combined_worldwide_df.Country.value_counts().shape[0]} countries from all across the world')

<i><p style="font-size:16px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">And finally here are the countries that are the slowly declining in participations in the Surveys Kaggle sends out each year. We have talked much about them in the past sections and do so some more further down this notebook as well. Many of them are from **Developed** and **Developed/Developing** countries, with the exception of those who are newcomers to the journey.

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">As we are half-way into this report, and have both the rising stars and the laggers before us, we might as well start thinking about what we could do to bridge the gap. Help find a mid-way for them, maybe have the risers collaborate with the laggers, that way both can learn from each other's experiences. Making the community and the survey reports that come out of these countries better with the coming years. We will see them appear further in the other classifications below where we can see them through other lenses.

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">**EDIT:** As mentioned in the [Revisions](#revisions) section, due to changes in the dataset, the table above has been impacted accordingly, and so has the relative positions of the countries in it has changed as mentioned in the above sections specific to each continent.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='active_or_not_on_kaggle'></a>

----------

### <div class="subsubsection_title"> 4. Which countries are NOT yet on Kaggle's world map?

### Having seen all of the metrics from the various countries active on Kaggle, it's time to look at those who are not and maybe list them here. This may help us understand if and when we can extend and include them in. Note that the list of countries shown here may vary from the official (and exhaustive) list, although we might be fairly close to it. This is just for illustration purposes and we don't need to be accurate (may contain mistakes encountered during the data collection and preprocessing stages) but they give us a ballpark figure and some starting point to work from.

We will see this from the perspective of each country's development status. You will see from the visuals what is meant by that. One thing about _Development Status_, it's clear mostly who the **Developed** and **Less/Under Developed** countries are more or less, although there is no such thing as a clear distinction between **Developed** and **Developing** countries. Different organisations can label some of the newly graduated **Developed** countries as **Developing** while others consider them as **Developed**. To avoid these confusions and ambiguity, in all such cases such countries have been assigned the _Development Status_ of **Developed/Developing**. There is still a chance to find a few countries with a slightly different _Development Status_ when going through other corpora of information about countries of the world. To find out more about what this means, please check out these links:

- Developed Countries
    - [Wikipedia: Developed country](https://en.wikipedia.org/wiki/Developed_country)
    - [Developed Countries List 2020](https://worldpopulationreview.com/country-rankings/developed-countries)
    - [Developing countries according to International Monetary Fund](https://en.wikipedia.org/wiki/Developing_country#Developing_countries_according_to_International_Monetary_Fund)
- Developing Countries
    - [Developing Countries](https://www.worlddata.info/developing-countries.php)
    - [Top 25 Developed and Developing Countries](https://www.investopedia.com/updates/top-developing-countries/)
- Less/Under developed Countries
    - [Least developed countries](https://en.wikipedia.org/wiki/Least_developed_countries)


<a id='countries_active_on_kaggle'></a>

----------

#### <div class="subsubsubsection_title">*a. Countries active on Kaggle* 

In [ ]:
def draw_treemap(dataframe: pd.DataFrame, columns: list = [], title: str = '[No title has been set]'):
    """
    Draw a treemap using the dataframe provided filtered by the list of columns, and the title.
    """
    figure = px.treemap(
        dataframe[columns], 
        title=title, 
        path=columns,
        width=1024 * 0.80,
        height=768 * 0.80
    )
    figure.show()
    
def create_pivot_table_with_total_count(dataframe: pd.DataFrame, index: list = ['Continent']) -> pd.DataFrame:
    """
    Create a pivot table based on some standard settings i.e. Margin enabled and named as 'Total count'.
    Also the values field is always 'Count' with aggfun of 'count'.
    The user can specific one or more index columns, when not specified 'Continent' is used.
    
    This will work specifically with dataframes containing the 'Count' and 'Continent' fields as a minimum.
    """    
    return dataframe.pivot_table('Count', index=index, \
                                  margins=True, margins_name='Total count', aggfunc='count')

In [ ]:
global_countries_list = pd.read_csv(f'{KAGGLE_PROCESSED_DATASET_PATH}/country_and_continent_info.csv')
global_countries_list = global_countries_list.rename(columns={'active_on_kaggle': 'Count'})
global_countries_list = global_countries_list.drop_duplicates(subset=['Country'], keep='first')
global_countries_list = global_countries_list.fillna(NOT_AVAILABLE)
global_countries_list = global_countries_list.replace({'Population': {NOT_AVAILABLE: 0}})

filter_not_present = (global_countries_list['Count'] == 0) & (global_countries_list['Continent'] != 'Antartica')
filter_active = (global_countries_list['Count'] == 1) & (global_countries_list['Continent'] != 'Antartica')

preferred_columns = ['Year', 'Development Status', 'Continent', 'Region', 'Country']
kaggle_combined_worldwide_df = kaggle_combined_worldwide_df.fillna(NOT_AVAILABLE)
kaggle_combined_worldwide_df = kaggle_combined_worldwide_df.sort_values(by=preferred_columns)

draw_treemap(kaggle_combined_worldwide_df, preferred_columns, 
             "Countries present on Kaggle (by Year, Development Status, & Continent)")

<i><p style="font-size:16px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">I love treemaps as they give both a birds-eye-view of things and we can also down-drill into the details if needed. The various hierarchies in the order of **Development Status**,  **Continent**, and **Countries** that fall there under can be seen for all those that are active on Kaggle and have responded to the surveys over the years. Do note that these are not the final list of active countries, these aree active countries that have also responded to the survey, with room for exceptions of participants from countries that may not be as active as the others but have still made an effort to complete the surveys across the years.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='countries_active_on_kaggle_by_dev_status'></a>

----------

#### <div class="subsubsubsection_title">*b. Breakdown of the number of countries active on Kaggle (by their Development Status)* 

In [ ]:
countries_active_on_kaggle_dev_status_df = create_pivot_table_with_total_count(global_countries_list[filter_active],
                                                                              index=['Development Status', 'Continent'])
countries_active_on_kaggle_dev_status_df = \
    sort_values_with_total_at_tail_end(countries_active_on_kaggle_dev_status_df, ('Count'))

prettify_display(countries_active_on_kaggle_dev_status_df, 
                 'Number of countries within each continent (all of them are active on Kaggle)')

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

In [ ]:
countries_active_on_kaggle_region_df = create_pivot_table_with_total_count(
    global_countries_list[filter_active], ['Region']).sort_values(by=[('Count')]
)


countries_active_on_kaggle_region_df = sort_values_with_total_at_tail_end(countries_active_on_kaggle_region_df, ('Count'))

prettify_display(countries_active_on_kaggle_region_df,
                 'Number of countries within each region (they are active on Kaggle)')

<i><p style="font-size:16px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">A pivot table helps with just this, bring together relevant details together and compute the required statistics about them. A breakdown of how many countries per **Development Status** per **Continent** is visible. We can see almost no **Less/Under Developed** countries are in the list (NOT present on Kaggle or did NOT responded to the surveys or both), and **Developed** and **Developed/Developing** countries dominate the table.
    
<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">In addition, the statistics on the number of countries active on Kaggle by regions in each of the continent is also interesting to know.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='countries_not_present_on_kaggle'></a>

----------

#### <div class="subsubsubsection_title">*c. Countries NOT present on Kaggle* 

In [ ]:
countries_not_present_on_kaggle_df = \
                        create_pivot_table_with_total_count(global_countries_list[filter_not_present])

draw_treemap(global_countries_list[filter_not_present], ['Development Status', 'Continent', 'Country'], 
             "Countries NOT present on Kaggle (by Development Status & Continent)")

<i><p style="font-size:16px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">Looking at the classification of the countries NOT present on Kaggle, just seems like a huge maze to solve. The size of the various maps show that the number of **Developed/Developing** and **Less/Under Developed** countries make up for the large part of the chunk. Only a small portion remains for **Developed** countries, which also means some amount of work would be needed in order to include/involve them and also sustain and support them thereafter. Also at the moment we are not receiving any signals from these regions of the world, and in the modern times we cannot and should not underestimate progress and advancements by ignoring lesser developed regions of the world.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='countries_not_active_on_kaggle_by_dev_status'></a>

----------

#### <div class="subsubsubsection_title">*d. Breakdown of the number of countries NOT present on Kaggle (by their Development Status)* 

In [ ]:
countries_not_present_on_kaggle_dev_status_df = \
        create_pivot_table_with_total_count(global_countries_list[filter_not_present], ['Development Status', 'Continent'])

countries_not_present_on_kaggle_dev_status_df = \
        sort_values_with_total_at_tail_end(countries_not_present_on_kaggle_dev_status_df, ('Count'))
    
prettify_display(countries_not_present_on_kaggle_dev_status_df,
                 'Number of countries within each continent (they are NOT present on Kaggle)')

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

In [ ]:
no_of_countries_by_the_region = create_pivot_table_with_total_count(
    global_countries_list[filter_not_present], ['Region']).sort_values(by=[('Count')]
)

no_of_countries_by_the_region = \
        sort_values_with_total_at_tail_end(no_of_countries_by_the_region, ('Count'))

prettify_display(no_of_countries_by_the_region,
                 'Number of countries within each region (they are NOT present on Kaggle)')

<i><p style="font-size:16px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">Yet another breakdown by the **Development Status** of the various countries across the **Continents**. As we saw from the treemap, the table above summarises it all with the statistics shown.
    
<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">In addition, the statistics on the number of countries NOT present on Kaggle by regions in each of the continent is also interesting to know.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='kaggle_wish_list_map'></a>

----------

#### <div class="subsubsubsection_title">*e. Map of countries that are NOT present or active on Kaggle (a wishlist)* 

In [ ]:
# Plotly Choropleth Maps reference: https://plotly.com/python/choropleth-maps/

TARGET_FIELD = 'Population'
SCALED_TARGET_FIELD = f'{TARGET_FIELD}_scaled'
sorted_and_not_present = global_countries_list.sort_values(by=[TARGET_FIELD], ascending=False)
sorted_and_not_present = sorted_and_not_present[filter_not_present]
sorted_and_not_present[SCALED_TARGET_FIELD] = sorted_and_not_present [TARGET_FIELD] / 1_000_000

not_active_on_kaggle_map = folium.Map(
    location=[0, 0], 
    tiles="stamenterrain",
    zoom_start=1.5
)

folium.Choropleth(
    geo_data='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json',
    name='choropleth',
    data=sorted_and_not_present,
    fill_color = 'OrRd',
    nan_fill_color='white',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    columns=['Country', SCALED_TARGET_FIELD],
    key_on='feature.properties.name',
    legend_name = f"{TARGET_FIELD} (in millions)",
).add_to(not_active_on_kaggle_map)

FastMarkerCluster(data=sorted_and_not_present[['latitude', 'longitude']]).add_to(not_active_on_kaggle_map)

folium.LayerControl().add_to(not_active_on_kaggle_map)

display(not_active_on_kaggle_map)

<i><p style="font-size:16px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">We have before us the future, the map of the world waiting for us to be explored. And who are we to say that those who have explored the world will not reach-out to those who haven't before the other sides reaches out to us. The colours and gradation in colours mark the countries and the population in them - sort of an indicative of which countries to reach out first or which ones might get left out. The non-coloured areas have already been covered but there is still more scope there too.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='comparing_countries_kaggle_status'></a>

----------

#### <div class="subsubsubsection_title">*f. Comparing statistics between countries that are active on Kaggle and those that are NOT* 

In [ ]:
def combine_two_dataframes(first_dataframe: pd.DataFrame, second_dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Append two dataframes one beneath the other and also do some cleanup.
        - remove the last line i.e. Total like rows
        - fill the NA/nan values with 0
    """

    combined_dataframe = pd.concat([first_dataframe, 
                                                 second_dataframe],  axis=1)
    combined_dataframe = combined_dataframe.fillna(0)
    combined_dataframe.columns = [ACTIVE_COL, NOT_PRESENT_COL]
    combined_dataframe[ACTIVE_COL] = combined_dataframe[ACTIVE_COL].astype(int)
    combined_dataframe[NOT_PRESENT_COL] = combined_dataframe[NOT_PRESENT_COL].astype(int)
    
    return combined_dataframe

countries_on_kaggle_combined_dev_status_df = combine_two_dataframes(countries_active_on_kaggle_dev_status_df, 
                                                   countries_not_present_on_kaggle_dev_status_df)

countries_on_kaggle_combined_dev_status_df = \
    sort_values_with_total_at_tail_end(countries_on_kaggle_combined_dev_status_df, (ACTIVE_COL))

prettify_display(countries_on_kaggle_combined_dev_status_df, 
                 f'Number of countries within each continent: {ACTIVE_COL} v/s {NOT_PRESENT_COL}')

<i><p style="font-size:16px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">Combining the two individual tables, we have seen so far creates this interesting table giving us a breakdown in a summarised fashion. The **red zeros** (there may be exceptions to this) and the **single digit** figures already indicate which areas across the various **Continents** need some attention and focus to balance the playing ground.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

In [ ]:
def draw_stacked_bar_chart(dataframe: pd.DataFrame, 
                           title: str = '[title not set]', 
                           ylabel_text: str = '[ylabel not set]'):
    """
    Draw a horizonal bar chart comparing the Active on Kaggle v/s Not Present on Kaggle country counts
    Also annotate the chart with the values and percentages for each bar.
    """
    plt.rcParams['figure.figsize'] = [15.0, 15.0]
    plt.rcParams['font.size'] = '26'
    sorted_dataframe = dataframe.sort_values(by=ACTIVE_COL)
    g = sorted_dataframe.plot(kind='barh', stacked=False, width=1)

    total_rows = sorted_dataframe[ACTIVE_COL].shape[0]

    totals = list((sorted_dataframe[ACTIVE_COL] + 
                   sorted_dataframe[NOT_PRESENT_COL]).values)

    margin = [0.0025, 0.0025]
    colours = [sns.color_palette("muted")[0], sns.color_palette("muted")[1]]
    for index, patch in enumerate(g.patches):
        percentage_value = 100 * (patch.get_width() / totals[index % len(totals)])
        percentage = ' {}  ({:.1f}%)'.format(
            int(patch.get_width()),
            percentage_value
        ) 
        x = patch.get_width() + (patch.get_width() * margin[index % 2])
        y = patch.get_y() + (patch.get_height() / 2) 
        g.annotate(percentage, (x, y), va='center', size=20, weight='bold',
                   color = colours[index >= total_rows])

    plt.legend(prop={'size': 24})
    for s in ['top', 'left', 'right','bottom']:
        g.spines[s].set_visible(False)

    g.spines['top'].set_visible(False)
    g.spines['right'].set_visible(False)
    g.spines['left'].set_visible(False)
    g.spines['bottom'].set_color('#DDDDDD')

    g.set_axisbelow(True)
    g.yaxis.grid(True, color='#EEEEEE')
    g.xaxis.grid(False)

    plt.title(title, weight='bold')
    plt.ylabel(ylabel_text)
    g.legend([ACTIVE_TEXT, NOT_PRESENT_TEXT], loc='lower right', ncol=3)

    plt.show()

In [ ]:
countries_active_on_kaggle_df = create_pivot_table_with_total_count(global_countries_list[filter_active])

countries_on_kaggle_combined_df = \
                     combine_two_dataframes(countries_active_on_kaggle_df, countries_not_present_on_kaggle_df)
countries_on_kaggle_combined_df = countries_on_kaggle_combined_df[:-1]

draw_stacked_bar_chart(
    countries_on_kaggle_combined_df,
    f'Countries per continent (values and percentages): \n{ACTIVE_TEXT} v/s {NOT_PRESENT_TEXT}',
    'Continents',
)

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">Looking at the above chart (graph/plot) we can see much opportunities before us, comparing the current situation and the future possibilities and prospects. There is a lot of participation from **Asia**, **Europe**, the **Americas** but looking at the number of non-particiapting countries (participation in Kaggle and Kaggle Surveys), makes me think that there may need to be some sort of outreach to those countries as well. Although the graph for **North America** looks skewed it's because of the smaller island nations that fall under the **North America** umbrella, and they could be a good adage to the future community. Otherwise the three big nations i.e. **Canada**, **USA** and **Mexico** have been participants since the beginning, although declining in participation. Same goes for **Oceania**, with an extra 30-odd countries under it, they are independent island countries around the continent of **Australiasia**. Also there are massive opportunities in **Africa**, given only **12.1%** of the continent is involved while a staggering **87.9%** of the continent is either unaware or not directly connecting with the Kaggle Data Science / Machine Learning communities, or may be even unable to connect (may not be a priority or do not have the necessary resources and support). **Note:** they are NOT present on Kaggle (or may not be participating) and hence NOT present in the Survey data since the beginning.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

In [ ]:
countries_active_on_kaggle_by_region_df = create_pivot_table_with_total_count(global_countries_list[filter_active], 'Region')
countries_not_present_on_kaggle_by_region_df = create_pivot_table_with_total_count(global_countries_list[filter_not_present], 'Region')

countries_not_present_on_kaggle_by_region_combined_df = \
                     combine_two_dataframes(countries_active_on_kaggle_by_region_df[:-1], 
                                            countries_not_present_on_kaggle_by_region_df[:-1])

draw_stacked_bar_chart(
    countries_not_present_on_kaggle_by_region_combined_df,
    f'Countries per region (values and percentages): \n{ACTIVE_TEXT} v/s {NOT_PRESENT_TEXT}',
    'Regions',
)

<i><p style="font-size:18px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">At a slightly lower-level i.e. at the regional-level per continent, there is high participation from **Asia (Near East)**, **West Europe**, the **Northern Africa**, and the **South America** but similarly there are a number of non-particiapting countries (participation in Kaggle and Kaggle Surveys), in the same regions (and neighbouring regions). Participations from the **North African** and **Middle-Eastern** regions are also noticable at the regional levels not just the country or continental levels -- even though this might be assumed to be an obvious cause-effect. Looking at the graph at the regional-level, we can see more countries that are NOT present on Kaggle (or may not be participating) and hence NOT present in the Survey data since the beginning.

<i><p style="font-size:21px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">These lenses help paint a better picture of the state of things with regards to the data collected via the **Kaggle Data Science / Machine Learning Survey** each year. We can think of it being something that's growing around certain edges but also retreating at others. The results gathered are still far from complete because we may be missing out the emerging and changing aspects of the world economy, progress and advancements. **Developed** countries do not necessary set the tone anymore, it's the **Developed/Developing**, **Developing** and **Less/Under Developed** countries that are setting the stage for the time to come.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

In [ ]:
def expand_and_interpolate_dateframe(dataframe: pd.DataFrame, 
                                     target_column: str, steps:int=5) -> pd.DataFrame:
    df_expanded = dataframe.copy()
    df_expanded = df_expanded.sort_values(by=['Year', SURVEY_RESPONSE_COLUMN], ascending=False)
    df_expanded.index = dataframe.index * steps
    last_index = df_expanded.index[-1] + steps
    df_expanded = df_expanded.reindex(range(last_index))
    df_expanded['Year'] = df_expanded['Year'].fillna(method='ffill')
    df_expanded[target_column] = df_expanded[target_column].fillna(method='ffill')
    df_expanded = df_expanded.interpolate()
    return df_expanded

def draw_horizontal_bar_chart(current_year):
    year_filter = dataframe['Year'] == current_year
    responses_for_a_year = dataframe[year_filter].sort_values(
        by=[SURVEY_RESPONSE_COLUMN], ascending=True).tail(MAX_ROWS
    )
    filter_non_zero_values = responses_for_a_year[SURVEY_RESPONSE_COLUMN] > 0
    responses_for_a_year = responses_for_a_year[filter_non_zero_values]
    responses_for_a_year = responses_for_a_year.reset_index(drop=True)
    responses_for_a_year[SURVEY_RESPONSE_COLUMN] = \
                   responses_for_a_year[SURVEY_RESPONSE_COLUMN].astype(int)        

    names = responses_for_a_year[BY_NAME_FIELD]
    survey_responses = responses_for_a_year[SURVEY_RESPONSE_COLUMN]

    plot_canvas.clear()
    for bar in plot_canvas.containers:
        bar.remove()
    plot_canvas.barh(names, survey_responses, 
                     tick_label=survey_responses.index,
                     color=[bar_colours[name] for name in names])

    margin = survey_responses.max() / 500
    for index, (value, name) in enumerate(zip(survey_responses, names)):
        plot_canvas.text(value - margin, index, name,  size=NAME_VALUE_SIZE, weight=NAME_TEXT_WEIGHT, ha='right', va='bottom') # Asia
        plot_canvas.text(value - margin, index, f'{value:,d}', size=NAME_VALUE_SIZE, ha='right',  va='top')                    # 12354.0
    
    nice_axes(plot_canvas)

    plot_canvas.text(1, 0.4, current_year, transform=plot_canvas.transAxes, size=YEAR_TEXT_SIZE, ha='right')
    plot_canvas.text(0, 1.06, SURVEY_RESPONSE_COLUMN, transform=plot_canvas.transAxes, size=SUBTITLE_TEXT_SIZE, color='#777777')
    plot_canvas.text(0, 1.12, f'Kaggle community by {BY_NAME_FIELD} between the years {FIRST_YEAR} and {LAST_YEAR}',
                transform=plot_canvas.transAxes, size=24, weight=TITLE_TEXT_WEIGHT, ha='left')
    
def init():
    plot_canvas.clear()
    nice_axes(plot_canvas)
    plot_canvas.set_ylim(.2, 6.8)
    
def nice_axes(plot_canvas):
    plot_canvas.set_facecolor('.8')
    plot_canvas.tick_params(labelsize=8, length=0)
    plot_canvas.grid(True, which='major', axis='x', linestyle='-', color='white')
    plot_canvas.set_axisbelow(True)
    [spine.set_visible(False) for spine in plot_canvas.spines.values()]
    
    plot_canvas.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    plot_canvas.xaxis.set_ticks_position('top')
    plot_canvas.tick_params(axis='x', colors='#777777', labelsize=12)
    plot_canvas.set_yticks([])
    plot_canvas.margins(0, 0.01)    
    plt.box(False)
    
def generate_colour():
    MAX_VALUE = 65535
    random_number = random.randint(0, MAX_VALUE//2) + random.randint(0, MAX_VALUE//3)
    hex_colour_value = str(hex(random_number))
    return f'#{hex_colour_value[2:]}'

<a id='racing_charts'></a>

----------

#### <div class="subsubsubsection_title">*g. A walk-through the years with survey responses from the continents, regions and countries of the Kaggle world* 


#### <div class="subsubsubsection_title">*Continents*

In [ ]:
responses_from_all_continents = kaggle_pivot_all_continents_summary_stats.drop(columns=[AVERAGE_AGGREGATED_GROWTH])
responses_from_all_continents = responses_from_all_continents.reset_index()
responses_from_all_continents = responses_from_all_continents[:-1]
responses_from_all_continents = pd.melt(responses_from_all_continents, id_vars=['Continent'], 
                                        var_name='Year', value_name=SURVEY_RESPONSE_COLUMN)
responses_from_all_continents = responses_from_all_continents.reset_index(drop=True)

dataframe = responses_from_all_continents
BY_NAME_FIELD = 'Continent'
MAX_ROWS = len(continents)

bar_colours = dict(zip(
     continents,
     plt.cm.Pastel1(range(len(continents)))
))

TITLE_TEXT_WEIGHT = 600
NAME_TEXT_WEIGHT = 600
YEAR_TEXT_SIZE = 46
SUBTITLE_TEXT_SIZE = 20

NAME_VALUE_SIZE = 18

figure, plot_canvas = plt.subplots(figsize=(20, 8), dpi=144, tight_layout=True)
animator = animation.FuncAnimation(figure, draw_horizontal_bar_chart, 
                                   init_func=init,
                                   frames=range(FIRST_YEAR, LAST_YEAR + 1),
                                   interval=750, repeat_delay=2000, repeat=False)
HTML(animator.to_jshtml())

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

#### <div class="subsubsubsection_title">*Regions*

In [ ]:
responses_from_all_regions = pd.melt(kaggle_combined_pivot_by_region, id_vars=['Region'], 
                                        var_name=[None, 'Year'], value_name=SURVEY_RESPONSE_COLUMN)
responses_from_all_regions = responses_from_all_regions.drop(columns=[None])

dataframe = responses_from_all_regions
BY_NAME_FIELD = 'Region'

regions = list(set(dataframe['Region'].values))
bar_colours = dict(zip(
    regions,
    plt.cm.Pastel1(range(len(regions)))
))
MAX_ROWS = len(regions)

NAME_VALUE_SIZE = 18

figure, plot_canvas = plt.subplots(figsize=(24, 12), dpi=144, tight_layout=True)
animator = animation.FuncAnimation(figure, draw_horizontal_bar_chart, 
                                   init_func=init,
                                   frames=range(FIRST_YEAR, LAST_YEAR + 1),
                                   interval=750, repeat_delay=2000, repeat=False)
HTML(animator.to_jshtml())

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

#### <div class="subsubsubsection_title">*Countries*

In [ ]:
responses_from_all_countries = responses_per_country_by_year.drop(columns=['latitude', 'longitude'])
responses_from_all_countries = pd.melt(responses_from_all_countries, 
                                       id_vars=['Country'], var_name=['Year'], value_name=SURVEY_RESPONSE_COLUMN)

dataframe = responses_from_all_countries

countries = list(set(dataframe['Country'].values))
bar_colours = dict(zip(
    countries,
    [generate_colour() for _ in  countries]
))

BY_NAME_FIELD = 'Country'
MAX_ROWS = len(countries) // 4  # top 10-15 countries only

NAME_VALUE_SIZE = 18

figure, plot_canvas = plt.subplots(figsize=(20, 14), dpi=144, tight_layout=True)
animator = animation.FuncAnimation(figure, draw_horizontal_bar_chart,
                                   init_func=init,
                                   frames=range(FIRST_YEAR, LAST_YEAR + 1),
                                   interval=750, repeat_delay=2000, repeat=False)

HTML(animator.to_jshtml())

<i><p style="font-size:21px; background-color: #FFF1D7; border: 2px solid black; margin: 20px; padding: 20px;">These racing bar charts of the continents, regions and countries representing the Kaggle community are simple interesting to the eyes, although we are not illustrating anything new that we haven't covered in the above sections of this notebook.

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='overallConclusion'></a>

----------

## <div class="subsection_title">Overall conclusion

<table border=1 bgcolor="#FF9F00"><tr><td>

<p style="font-size:16px">To conclude the discussions after brief conjectures in the two sections above namely, the <a href=#questions>Answering questions with the dataset</a> section and it's subsections, it's time to make some concluding points after our observations.</p>

<p style="font-size:16px">We are not perfect and neither is the data we handle on a day-to-day basis. But even if there are/were mistakes in the above presentations, I truly hope that we can still get the idea, or <i><b>the gist of the message and insights above can serve as a template for it. I'm sure further investigation and tweaks will help us fine-tune our analysis and reports. </b></i> One thing we must try to find out, if the limitations mentioned in this report are also helping find out the biases in the the data and how our interpretations and explainations make this clear and transparent. I'm hoping my conclusions in this whole notebook are more correct than wrong and that mistakes in the data-processing or data analysis steps have not led to omissions, mistakes or errors in the conclusions and other outcomes. I hope that the tables and charts provided in this notebook provide the necessary ingredients to perform the next steps (see the <a href=#revisions>Revisions section</a> for more on this).</p>

<p style="font-size:16px">Of the many things we can see, we can see particularly which regions of the world and specifically which countries of the world are responsive and are happy to spend time to contribute back and taking more and more initiatives. They are becoming the next generation of catalyst bringing a change and also speaking out their views and opinions on what they do and why they do it. You must have already read about them in this section <a href=#growing_and_retreating>3. Which countries are growing in participations and which ones are retreating?</a> (subsections <a href=#growing_in_participation>a. Countries growing in participation (rising stars)</a> and <a href=#retreating>b. Countries retreating (mid-aged and veterans)</a>). There are a few notebook submissions which may also be talking about this and also related topics which would be good to follow-through.</p>
    
<p style="font-size:16px">There are still many countries in various regions (continents) of the world, that may still NOT be part of the Data Science / Machine Learning big-picture and they could be reached out to extend the community. Particularly the <b>Less/Under Developed</b> countries that has been highlighted in the subsection <a href=#countries_not_active_on_kaggle_by_dev_status>d. Breakdown of the number of countries NOT present on Kaggle (by their Development Status)</a> and <a href='#comparing_countries_kaggle_status'>e. Comparing statistics between countries that are active on Kaggle and those that are NOT</a>. But more than growth, we may wish to focus on sustaining and supporting the existing members of the community. Given we have had many challenging months behind us, coming back up to our normal or original states may take some help and support from outside.</p>

<p style="font-size:16px"><i><b>The community is not just the ones who spend time on the <a href="https://www.kaggle.com/competitions">Competitions</a>, <a href="https://www.kaggle.com/discussion">Discussion forums</a> and submit <a href="https://www.kaggle.com/notebooks">Notebooks</a> and <a href="https://www.kaggle.com/datasets">Datasets</a> regularly but also the ones who directly and indirectly are contributing to notebooks and reports like  <a href="https://www.kaggle.com/kaggle/kaggle-survey-2017/notebooks">Kaggle Survey 2017 Noteobook</a> | <a href="https://www.kaggle.com/kaggle/kaggle-survey-2018/notebooks">Kaggle Survey 2018 Noteobook</a> | <a href="https://www.kaggle.com/kaggle-survey-2019">State of Data Science and Machine Learning 2019</a>  |  <a href="https://www.kaggle.com/kaggle-survey-2020">State of Data Science and Machine Learning 2020</a> from the years 2017, 2018, 2019 and 2020 respectively.</b></i></p>
    
<p style="font-size:16px">I would also like to acknowledge that Kaggle is doing a lot to reach out and support existing members as this I know as I have received invites to <i><b>Grants and Scholarships (see <a href="https://www.kaggle.com/bipoc-grant-application">the  "Kaggle BIPOC Grant Application" site</a>) should I need support to continue with my Data Science / Machine Learning journey</b></i>. And this among the other things that are done to keep us engaged and involved, we keep learning and getting rewarded and incentivised to stay tuned with the Kaggle forums and initiatives.

<p style="font-size:18px">Once again thanks to all those who provided such amazing notebooks, visualisations and useful datasets that my work of preparing these notebooks and datasets became easier, and I could focus more on answering the questions I have raised above.</p>
</td></tr></table>

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>

<a id='credits'></a>

----------

## <div class="subsection_title">Credit and thanks

I'd like to give credit and also thanks to those who have shared their amazing notebooks in the open for everyone to learn from. I have been inspired by the ideas and took them over into my notebook and made further improvements to them:

- [docxian](https://www.kaggle.com/docxian) for the treemaps idea (see source notebook [Interactive Visualizations using Treemaps](https://www.kaggle.com/docxian/interactive-visualizations-using-treemaps)).
- [chandramanaha](https://www.kaggle.com/chandramanaha) for the line plots-with text annotations in them (see source notebook [KYC-Know Your Community!](https://www.kaggle.com/chandramanaha/kyc-know-your-community)).
- Thanks to [Parul Pandey](https://www.kaggle.com/parulpandey) for her notebook with the racing bar charts in [💰Who earned the most in Sports in 2020🏆?](https://www.kaggle.com/parulpandey/who-earned-the-most-in-sports-in-2020#7.-Racing-Bar-chart) and also to [Pratap Vardhan](https://medium.com/@PratapVardhan?source=post_page-----8e687a5c8a41--------------------------------) for his TDS post on [Bar Chart Race in Python with Matplotlib](https://towardsdatascience.com/bar-chart-race-in-python-with-matplotlib-8e687a5c8a41). And also [Ted Petrou](https://petrou-theodore.medium.com/?source=post_page-----477ed1590096--------------------------------) who wrote this post [Creating a Bar Chart Race Animation in Python with Matplotlib](https://medium.com/dunder-data/create-a-bar-chart-race-animation-in-python-with-matplotlib-477ed1590096).
- I have also thanked those who shared datasets or notebooks to prepare datasets for this kernel in the credit sections (bottom of the notebook) of these notebooks:
    - [Kaggle Machine Learning & Data Science (data-prep)](https://www.kaggle.com/neomatrix369/kaggle-machine-learning-data-science-data-prep/)
    - [ChaiEDA sessions: World-bank-data 1960-2016 (data-prep)](https://www.kaggle.com/neomatrix369/chaieda-world-bank-data-1960-2016-data-prep)
- _and many others who has helped, inspired and have been catalysts in the process._

### <div class="subsection_title">Prequels/sequels

- [Kaggle Machine Learning & Data Science (data-prep)](https://www.kaggle.com/neomatrix369/kaggle-machine-learning-data-science-data-prep/) | [Extended Dataset](https://www.kaggle.com/neomatrix369/kaggle-machine-learning-data-science-survey-ext) | [Additional Dataset](https://www.kaggle.com/neomatrix369/world-bank-data-1960-to-2016-extended)
- **Kaggle Global Outreach (analysis)**

<a href='#ToC'><span class="label label-info" style="font-size: 125%">Back to Table of Contents</span></a>